**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-z4rt64rr
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-z4rt64rr
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-z4rt64rr did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-z4rt64rr did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,f0b2cd14db66739d234bcfd3bb032a9777c59099,b70f4b9ecf75a6f458b2e32e5cc3de7f327c7928dd9e62...,2014-03-11 15:00:00+00:00,2014-03-11 15:15:00+00:00,720,1.8,1.703108e+10,1.703128e+10,8.0,28.0,...,0.0,8.45,Cash,0118 - Godfrey S.Awir,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.879255,-87.642649,POINT (-87.642648998 41.8792550844)
1,57c0daa3b1452354d9c4f30a79e138d649e2936b,e1a41c8d6d3b0e4aa884ccb8a99f70075f76d82a7f1f5f...,2014-03-05 18:30:00+00:00,2014-03-05 19:00:00+00:00,1200,3.3,1.703184e+10,1.703107e+10,32.0,7.0,...,0.0,14.20,Credit Card,1085 - N and W Cab Co,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.916473,-87.641184,POINT (-87.6411836567 41.9164733164)
2,15433332870c926e726d2e5ca01a068b9431e11e,7677e1d56736b3d9dbca24713dd72384d1fc24e78f3b64...,2014-09-19 17:00:00+00:00,2014-09-19 17:15:00+00:00,240,1.6,NaN,NaN,NaN,NaN,...,0.0,6.65,Cash,1247 - 72807 Daniel Ayertey,NaN,NaN,None,NaN,NaN,None
3,da619a68871fecfb4c07ccbf816e1281a382020b,7677e1d56736b3d9dbca24713dd72384d1fc24e78f3b64...,2014-08-04 09:45:00+00:00,2014-08-04 10:00:00+00:00,600,1.2,NaN,NaN,NaN,NaN,...,0.0,7.25,Cash,1247 - 72807 Daniel Ayertey,NaN,NaN,None,NaN,NaN,None
4,c6c652f10dd729b4541ef2e379992db0290d38ed,7677e1d56736b3d9dbca24713dd72384d1fc24e78f3b64...,2014-08-20 15:45:00+00:00,2014-08-20 16:00:00+00:00,480,1.1,NaN,NaN,NaN,NaN,...,1.5,8.15,Cash,1247 - 72807 Daniel Ayertey,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                             AVG(num_trips) OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-03-24,30124.285714
1,2016-01-03,25915.000000
2,2016-01-14,30010.000000
3,2016-03-05,32154.714286
4,2016-03-26,29686.285714


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [4]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER (partition by pickup_community_area ORDER BY trip_start_timestamp) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,18.0,2013-10-03 08:15:00+00:00,2013-10-03 08:15:00+00:00,1
1,5.0,2013-10-03 01:00:00+00:00,2013-10-03 01:15:00+00:00,1
2,5.0,2013-10-03 10:15:00+00:00,2013-10-03 10:45:00+00:00,2
3,5.0,2013-10-03 15:00:00+00:00,2013-10-03 15:15:00+00:00,3
4,5.0,2013-10-03 16:30:00+00:00,2013-10-03 16:45:00+00:00,4


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [5]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0203af03b9b42f1f43acb2a6d9a633ca8983314d147309...,2013-10-03 07:45:00+00:00,2013-10-03 07:45:00+00:00,NaN
1,6acd828b5cde8b4529317e8eda5b4fb9663fa81d1143c9...,2013-10-03 13:45:00+00:00,2013-10-03 14:15:00+00:00,NaN
2,7f8be35bf085d7490245401fe27b274624245d8e76ae11...,2013-10-03 13:30:00+00:00,2013-10-03 13:30:00+00:00,NaN
3,074a32daa8f72bd629b2b9f17a5a90ce63b8e6f0aae58f...,2013-10-03 07:45:00+00:00,2013-10-03 07:45:00+00:00,NaN
4,124d3882c5d0adc22a876f6a392845233eb5b2cfb49ef1...,2013-10-03 04:30:00+00:00,2013-10-03 04:15:00+00:00,NaN


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*